In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression

In [ ]:
class File:
    def __init__(self, direction, duty_cycle, repetition):
        self.direction = direction
        self.duty_cycle = duty_cycle
        self.repetition = repetition
        self.name = self.generate_name()
    
    def generate_name(self):
        name = "02/{direction:s}_{duty_cycle:.2f}_{repetition:02d}.csv"
        return name.format(direction = self.direction, duty_cycle=self.duty_cycle, repetition=self.repetition)

def filenames(min, max, step, repetitions):
    files = []
    i = min
    while i <= max:
        for j in range(repetitions):
            files.append(File(direction = "f", duty_cycle = i, repetition = j))
            files.append(File(direction = "b", duty_cycle = i, repetition = j))
        i += step
    return files

In [ ]:
def get_data_from_file(name):
    with open(name) as csv_file:
        df = pd.read_csv(csv_file, header=None, names=["Time", "Distance"])
        start_time = int(df.loc[0]["Time"])
        start_distance = float(df.loc[0]["Distance"])
        df *= [1, -1]
        df += [-start_time, start_distance]
    return df

In [ ]:
files = filenames(0.3, 1.04, 0.05, 3)
l = []
for file in files:
    # Get data
    df = get_data_from_file(file.name)
    df = df.tail(100)
    x = df["Time"].values.reshape(-1, 1)
    y = df["Distance"].values.reshape(-1, 1)

    # Perform linear regression
    regr = LinearRegression()
    regr.fit(x, y)
    velocity = regr.coef_[0][0]*1000

    print(file.direction, file.duty_cycle)
    print(velocity)

    # # Plot the results
    # plt.plot(x, y)
    # plt.plot(x, regr.predict(x))
    # plt.show()

    # Prepare data for global analysis
    if file.direction == "f":
        l.append([file.duty_cycle, velocity])
    else:
        l.append([file.duty_cycle, -velocity])

    # velocity = regr.predict() - regr.predict()
    # print(velocity*0.001, "m/s")

In [ ]:
df = pd.DataFrame(l, columns=["Duty cycle", "Velocity"])
# print(df)
x = df["Duty cycle"].values.reshape(-1, 1)
y = df["Velocity"].values.reshape(-1, 1)
# y = np.exp(y)
# y = y ** 2

df["Mean"] = df.groupby("Duty cycle")["Velocity"].transform("mean")
y2 = df["Mean"].values.reshape(-1, 1)

y3 = y2 ** 4
y3 = np.exp(y3)

print(df)

regr = LinearRegression()
regr.fit(x, y3)

# plt.scatter(x, y)
plt.scatter(x, y3)
plt.plot(x, regr.predict(x))
plt.show()

coefs = regr.predict(np.array([0, 1]).reshape(-1, 1))
coefs = [coefs[1][0]-coefs[0][0], coefs[0][0]]
print(coefs)

In [ ]:
# Somewhat accurate prediction of velocity based on duty cycle
def velocity_prediction(duty_cycle):
    return np.power(np.log(0.3474*duty_cycle+0.9077), 0.25)

# Wheel circumference: ~25 cm
def rpm_prediction(duty_cycle):
    return 60*velocity_prediction(duty_cycle)/0.25

plt.scatter(x, y2)
plt.plot(x, velocity_prediction(x))
plt.show()
for i in range(30, 101, 5):
    print(0.01*i, rpm_prediction(0.01*i))